# Setup

In [136]:
!pip -q install openai
!pip -q install langchain
!pip -q install langchain_openai # >= 0.1.0
!pip -q install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 3.3 MB/s eta 0:00:00


In [139]:
#API KEY 저장을 위한 os 라이브러리 호출
import os
from google.colab import userdata

#OPENAI API키 저장
#API KEY 발급 페이지: https://platform.openai.com/docs/guides/gpt/completions-api
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Text Splitters

- Document Loader로 읽은 문서를 작은 문서 조각(Chunk)로 자르는 도구

- LLM은 토큰 제한(Context Window)가 있기 때문에 전체 문서를 참고할 수 없고 작은 문서 조각으로 나눠서 사용자의 질문과 "의미적"으로 유사한 문서 조각을 찾고 이를 기반으로 LLM이 답변을 생성하기 만들기 위함


## Split by chaaracter

- `CharacterTextSplitter`

- 조각을 나누는 기준을 하나만 사용하여 chunk_size를 지키게 분할

- 기준을 하나만 사용하므로 chunk_size을 넘어 갈 수 있음

- https://python.langchain.com/docs/modules/data_connection/document_transformers/character_text_splitter

In [9]:
# LangChain 공식 도움말 예제 파일
# https://github.com/hwchase17/chat-your-data/blob/master/state_of_the_union.txt

# file open and read for url
import urllib.request

with urllib.request.urlopen("https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt") as f:
    state_of_the_union = f.read().decode("utf-8")


In [6]:
from langchain.text_splitter import CharacterTextSplitter

In [68]:
chunk_size = 300
chunk_overlap = 100

text_splitter = CharacterTextSplitter(
    separator="\n\n", # 조각을 나눌 구분자, chunk_size보다 우선
    # 조각의 최대 크기
    chunk_size=chunk_size,
    # 조각과 조각이 겹치는 정도, 문장단위로 겹칠지 안 겹칠지 판단
    # 값이 작고 겹쳐야 하는 문장이 길면 겹치지 않을 수 있음
    # 200으로 늘려서 비교 필요
    chunk_overlap=chunk_overlap,
    length_function=len, # chunk_size를 측정할 함수, 여기서는 영문 글자수가 됨
    is_separator_regex=False, # 구분자가 정규표현식인가?
)

In [64]:
# Document 객체로 분할
texts = text_splitter.create_documents([state_of_the_union])

# 그냥 plain text로 분할
# texts = text_splitter.split_text(state_of_the_union)

In [65]:
texts

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again.'),
 Document(page_content='Last year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution.'),
 Document(page_content='With a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.'),
 Document(page_content='And with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.

In [66]:
print(texts[0].page_content)
print('-------------------')
print(texts[1].page_content)
print('-------------------')
print(texts[2].page_content)
print('-------------------')
print(texts[3].page_content)
print('-------------------')
print(texts[4].page_content)
print('-------------------')
print(texts[5].page_content)


Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again.
-------------------
Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution.
-------------------
With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny.
-------------------
And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.
-------------------
He thought he could roll into Ukra

In [112]:
# 모든 조각이 chunk_size보다 작은가?
[len(text.page_content) for text in texts if len(text.page_content) > chunk_size]

[]

### 한글 PDF 실습

In [ ]:
# 실습: 한글 pdf파일을 읽고 자르기

# 가끔 gdown 에러 날 때
!pip install gdown==4.6.0

!gdown 15zGjdjHS279Byb_06P1qugToEzS1ivg4

In [73]:
!pip -q install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 4.6 MB/s eta 0:00:00


In [71]:
from langchain_community.document_loaders import PyPDFLoader

In [113]:
pdf_loader = PyPDFLoader("초거대 언어모델 연구 동향_업스테이지_6000원.pdf")

# pdf 파일 페이지별로 잘라서 읽어옴
pages = pdf_loader.load_and_split()

In [ ]:
pages[:3]

In [116]:
text_splitter = CharacterTextSplitter(
    separator="\n", # 조각을 나눌 구분자 \n\n로 하고 어떻게 잘리는지 확인
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len, # chunk_size를 측정할 함수, 여기서는 영문 글자수가 됨
)

In [117]:
# 이미 pages가 Document의 리스트라서 pages를 그대로 넘기고
# split_documents를 사용
texts = text_splitter.split_documents(pages)

In [118]:
print(texts[0].page_content)
print('-------------------------')
print(texts[1].page_content)

8특집원고  초거대 언어모델 연구 동향초거대 언어모델 연구 동향
업스테이지   박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
ChatGPT 1)와 같은 초거대 언어모델 (Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
던 다양한 자연언어처리 하위 분야들이 하나의 모델로 처리되고 있으며 , 태스크 수렴 현상 (Converge) 이 
발생하고 있다 . 즉 하나의 LLM으로 번역 , 요약 , 질의
응답 , 형태소분석 등의 작업을 모두 처리할 수 있게 
되었다 . 프롬프트 (Prompt) 를 어떻게 모델에게 입력하
느냐에 따라서 LLM의 다양한 능력들이 창발되고 , 이
에 따라 사용자의 목적에 맞는 출력을 생성하는 패러다임을 맞이하게 되었다 [1].
LLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 
발전을 이루고 있다 . 이러한 발전은 몇 가지 주요한 
요인에 기반하고 있으며 , 이 요인들은 현대 자연언어
처리 (Natural Language Processing, NLP) 연구의 핵심
적인 추세로 간주된다 . 첫째로 , 데이터의 양적 확대는 
무시할 수 없는 중요한 요인이다 . 디지털화의 선도로 , 
텍스트 데이터의 양이 기하급수적으로 증가하였고 , 
이는 연구의 질적 변화를 가져왔다 . 대규모 코퍼스의 
활용은 LLM의 일반화 능력을 향상시키며 , 다양한 맥
락과 주제에 대한 깊은 학습을 가능하게 한다 . 둘째
로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정
적이었다 . 특히 , Graphics Processing Unit (GPU) 및 
Tensor Processing Unit (TPU) 와 같은 고성능 병렬 처
리 하드웨어의 개발은 모델 학습에 있어 병목 현상을 크게 완화시켰다 . 이로 인해 연구자들은 모델의 복잡
성을 키우고 , 더욱 깊은 신경망 구조를 탐구할 수 있
게 되었다 . 셋째 , 알고리즘 및 기술의 발전은 LLM의 
성능 향상을 주도하였다 . Attention 및 Transformer
--

In [119]:
# 76개 조각으로 분할
print(len(texts))

# 모든 조각이 chunk_size보다 작은가?
print([len(text.page_content) for text in texts])

76
[997, 954, 607, 931, 973, 965, 219, 977, 944, 979, 552, 978, 992, 972, 493, 985, 990, 972, 342, 971, 987, 970, 472, 963, 966, 965, 516, 994, 950, 978, 600, 964, 983, 948, 485, 988, 980, 452, 989, 962, 965, 689, 987, 944, 990, 983, 693, 859, 998, 946, 948, 979, 748, 921, 925, 988, 1000, 945, 686, 937, 981, 997, 971, 959, 482, 973, 233, 954, 944, 967, 963, 762, 860, 975, 999, 251]


## Recursively split by character

- `RecursiveCharacterTextSplitter`

- 조각을 나누는 기준을 여러개 사용하여 여러 기준으로 chunk_size 제한을 최대한 지키게 반복적으로 분할

- 일반 텍스트에 권장되는 텍스트 분할기

- 기본 분할 기준 목록은 ["\n\n", "\n", " ", ""]을 사용해서 조각이 충분히 작아질 때까지 순서대로 분할을 시도입니다.

- 일반적으로 의미적으로 가장 연관성이 강한 텍스트 조각인 것처럼 보이는 모든 단락(그리고 문장, 단어)을 가능한 한 길게 유지하려는 효과


- https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter


In [106]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [107]:
text_splitter = RecursiveCharacterTextSplitter(
    # separators = [] # 필요하면 설정
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False,
)

In [108]:
texts = text_splitter.create_documents([state_of_the_union])


In [110]:
print(texts[0].page_content)
print('-------------------')
print(texts[1].page_content)
print('-------------------')
print(texts[2].page_content)
print('-------------------')
print(texts[3].page_content)
print('-------------------')
print(texts[4].page_content)
print('-------------------')
print(texts[5].page_content)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again.
-------------------
Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution.
-------------------
With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny.
-------------------
And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.
-------------------
He thought he could roll into Ukra

In [111]:
# 모든 조각이 chunk_size보다 작은가?
[len(text.page_content) for text in texts if len(text.page_content) > chunk_size]

[]

### 한글 PDF 실습

In [126]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=10,
    length_function=len, # chunk_size를 측정할 함수, 여기서는 영문 글자수가 됨
)

In [127]:
texts = text_splitter.split_documents(pages)

In [128]:
print(texts[0].page_content)
print('-------------------------')
print(texts[1].page_content)

8특집원고  초거대 언어모델 연구 동향초거대 언어모델 연구 동향
업스테이지   박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
ChatGPT 1)와 같은 초거대 언어모델 (Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
던 다양한 자연언어처리 하위 분야들이 하나의 모델로 처리되고 있으며 , 태스크 수렴 현상 (Converge) 이 
발생하고 있다 . 즉 하나의 LLM으로 번역 , 요약 , 질의
응답 , 형태소분석 등의 작업을 모두 처리할 수 있게
-------------------------
되었다 . 프롬프트 (Prompt) 를 어떻게 모델에게 입력하
느냐에 따라서 LLM의 다양한 능력들이 창발되고 , 이
에 따라 사용자의 목적에 맞는 출력을 생성하는 패러다임을 맞이하게 되었다 [1].
LLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 
발전을 이루고 있다 . 이러한 발전은 몇 가지 주요한 
요인에 기반하고 있으며 , 이 요인들은 현대 자연언어
처리 (Natural Language Processing, NLP) 연구의 핵심
적인 추세로 간주된다 . 첫째로 , 데이터의 양적 확대는


In [129]:
# 76개 조각으로 분할
print(len(texts))

# 모든 조각이 chunk_size보다 작은가?
print([len(text.page_content) for text in texts])

218
[279, 279, 272, 273, 285, 277, 245, 265, 36, 290, 284, 294, 270, 287, 271, 287, 272, 266, 75, 240, 297, 298, 269, 262, 258, 272, 254, 281, 292, 218, 286, 298, 279, 295, 163, 298, 266, 289, 276, 268, 122, 294, 298, 287, 289, 279, 286, 273, 282, 284, 215, 245, 278, 265, 245, 290, 273, 284, 297, 284, 271, 138, 288, 267, 281, 265, 278, 294, 287, 278, 282, 285, 33, 269, 287, 274, 255, 288, 286, 279, 275, 296, 262, 210, 293, 269, 297, 265, 295, 286, 267, 277, 276, 276, 30, 276, 273, 296, 297, 270, 264, 298, 130, 252, 265, 279, 287, 281, 275, 271, 269, 242, 294, 285, 107, 280, 284, 268, 260, 257, 293, 234, 296, 259, 259, 243, 263, 272, 274, 206, 288, 283, 284, 248, 296, 268, 298, 285, 280, 253, 246, 272, 272, 264, 226, 287, 274, 183, 295, 298, 289, 33, 272, 285, 297, 289, 263, 270, 278, 285, 282, 295, 274, 250, 270, 264, 53, 288, 295, 297, 51, 289, 239, 256, 257, 299, 220, 273, 281, 225, 290, 272, 297, 246, 264, 216, 271, 286, 296, 165, 266, 254, 259, 280, 276, 236, 287, 262, 189, 281, 26

## Split code

- `RecursiveCharacterTextSplitter.from_language()`

- 여러 프로그램 언어를 지원하여 코드를 분할

- https://python.langchain.com/docs/modules/data_connection/document_transformers/code_splitter

In [97]:
from langchain.text_splitter import Language

In [98]:
# Full list of supported languages
[e.value for e in Language]

['cpp',
 'go',
 'java',
 'kotlin',
 'js',
 'ts',
 'php',
 'proto',
 'python',
 'rst',
 'ruby',
 'rust',
 'scala',
 'swift',
 'markdown',
 'latex',
 'html',
 'sol',
 'csharp',
 'cobol']

### python

In [101]:
# You can also see the separators used for a given language
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)

['\nclass ', '\ndef ', '\n\tdef ', '\n\n', '\n', ' ', '']

In [102]:
PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Call the function
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(page_content='def hello_world():\n    print("Hello, World!")'),
 Document(page_content='# Call the function\nhello_world()')]

### html

In [130]:
RecursiveCharacterTextSplitter.get_separators_for_language(Language.HTML)

['<body',
 '<div',
 '<p',
 '<br',
 '<li',
 '<h1',
 '<h2',
 '<h3',
 '<h4',
 '<h5',
 '<h6',
 '<span',
 '<table',
 '<tr',
 '<td',
 '<th',
 '<ul',
 '<ol',
 '<header',
 '<footer',
 '<nav',
 '<head',
 '<style',
 '<script',
 '<meta',
 '<title',
 '']

In [103]:
html_text = """
<!DOCTYPE html>
<html>
    <head>
        <title>🦜️🔗 LangChain</title>
        <style>
            body {
                font-family: Arial, sans-serif;
            }
            h1 {
                color: darkblue;
            }
        </style>
    </head>
    <body>
        <div>
            <h1>🦜️🔗 LangChain</h1>
            <p>⚡ Building applications with LLMs through composability ⚡</p>
        </div>
        <div>
            As an open-source project in a rapidly developing field, we are extremely open to contributions.
        </div>
    </body>
</html>
"""

In [134]:
html_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.HTML, chunk_size=100, chunk_overlap=0
)
html_docs = html_splitter.create_documents([html_text])
html_docs


[Document(page_content='<!DOCTYPE html>\n<html>'),
 Document(page_content='<head>\n        <title>🦜️🔗 LangChain</title>'),
 Document(page_content='<style>\n            body {\n                font-family: Arial, sans-serif;\n            }'),
 Document(page_content='h1 {\n                color: darkblue;\n            }\n        </style>\n    </head>'),
 Document(page_content='<body>'),
 Document(page_content='<div>\n            <h1>🦜️🔗 LangChain</h1>'),
 Document(page_content='<p>⚡ Building applications with LLMs through composability ⚡</p>\n        </div>'),
 Document(page_content='<div>\n            As an open-source project in a rapidly developing field, we are extremely open to'),
 Document(page_content='contributions.\n        </div>\n    </body>\n</html>')]

## Split by tokens

- LLM의 토큰 제한에 정확히 맞추기 위해 토큰수 기준으로 문서를 분할

- 당연하지만 문서에서 토큰을 계산할 때는 언어 모델에 사용된 것과 동일한 토큰화기를 사용해야 함

### tiktoken




In [144]:
# OpenAI 모델들이 사용하는 토크나이저 설치
!pip install tiktoken

#### `.from_tiktoken_encoder()`

In [221]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
texts = text_splitter.split_text(state_of_the_union)

In [230]:
# 단어수(공백기준)
[len(text.split(' ')) for text in texts]

# 토큰 수는?

[697, 767, 689, 634, 692, 669, 677, 704, 658, 383]

- 모델에 따라 다음과 같은 인코딩 방식이 있음

- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

| Encoding name | OpenAI models |
|---------------|---------------|
| cl100k_base   | gpt-4, gpt-3.5-turbo, text-embedding-ada-002 |
| p50k_base     | Codex models, text-davinci-002, text-davinci-003 |
| r50k_base (or gpt2) | GPT-3 models like davinci |

In [231]:
import tiktoken

tokenizer = tiktoken.get_encoding("r50k_base")
# tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [233]:
# encoding
state_of_the_union[:5], tokenizer.encode(state_of_the_union[:5])

('Madam', [18454, 321])

In [234]:
# decoding
[
    tokenizer.decode_single_token_bytes(token)
        for token in tokenizer.encode(state_of_the_union[:5])
]

[b'Mad', b'am']

In [235]:
def num_tokens_from_string(string: str, encoding_name: str="r50k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [236]:
num_tokens_from_string(state_of_the_union[:5])

2

In [237]:
# 단어수(공백기준), 토큰수
[(len(text.split(' ')), num_tokens_from_string(text)) for text in texts]

[(697, 977),
 (767, 994),
 (689, 981),
 (634, 981),
 (692, 980),
 (669, 973),
 (677, 983),
 (704, 996),
 (658, 963),
 (383, 571)]

#### 직접 하기

In [227]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    length_function=num_tokens_from_string,
)

texts = text_splitter.split_text(state_of_the_union)

In [228]:
# 단어수(공백기준), 토큰수
[(len(text.split(' ')), num_tokens_from_string(text)) for text in texts]

[(697, 977),
 (767, 994),
 (689, 981),
 (634, 981),
 (692, 980),
 (669, 973),
 (677, 983),
 (704, 996),
 (658, 963),
 (383, 571)]

### Hugging Face tokenizer

In [ ]:
from transformers import GPT2TokenizerFast

In [145]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [171]:
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=100, chunk_overlap=0
)

texts = text_splitter.split_text(state_of_the_union)

In [176]:
# 단어 기준
print(len(texts[0].split(' ')))
print(texts[0])

65
Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution.


In [177]:
# 토큰 기준
print(len(tokenizer.tokenize(texts[0])))
print(tokenizer.tokenize(texts[0]))

94
['Mad', 'am', 'ĠSpeaker', ',', 'ĠMad', 'am', 'ĠVice', 'ĠPresident', ',', 'Ġour', 'ĠFirst', 'ĠLady', 'Ġand', 'ĠSecond', 'ĠGentleman', '.', 'ĠMembers', 'Ġof', 'ĠCongress', 'Ġand', 'Ġthe', 'ĠCabinet', '.', 'ĠJust', 'ices', 'Ġof', 'Ġthe', 'ĠSupreme', 'ĠCourt', '.', 'ĠMy', 'Ġfellow', 'ĠAmericans', '.', 'Ġ', 'Ġ', 'Ċ', 'Ċ', 'Last', 'Ġyear', 'ĠCO', 'VID', '-', '19', 'Ġkept', 'Ġus', 'Ġapart', '.', 'ĠThis', 'Ġyear', 'Ġwe', 'Ġare', 'Ġfinally', 'Ġtogether', 'Ġagain', '.', 'Ġ', 'Ċ', 'Ċ', 'Tonight', ',', 'Ġwe', 'Ġmeet', 'Ġas', 'ĠDemocrats', 'ĠRepublicans', 'Ġand', 'ĠInd', 'epend', 'ents', '.', 'ĠBut', 'Ġmost', 'Ġimportantly', 'Ġas', 'ĠAmericans', '.', 'Ġ', 'Ċ', 'Ċ', 'With', 'Ġa', 'Ġduty', 'Ġto', 'Ġone', 'Ġanother', 'Ġto', 'Ġthe', 'ĠAmerican', 'Ġpeople', 'Ġto', 'Ġthe', 'ĠConstitution', '.']


In [174]:
# 모든 조각이 chunk_size보다 작은가?
print([len(tokenizer.tokenize(texts[0])) for text in texts])

[94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94]



## Semantic Chunking

- `SemanticChunker`

- LLM을 사용하여 의미적 유사성을 기준으로 텍스트를 분할

- https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker

In [137]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
# from langchain_openai import ChatOpenAI # for >= 0.1.0

In [140]:
text_splitter = SemanticChunker(OpenAIEmbeddings())

In [142]:
texts = text_splitter.create_documents([state_of_the_union])


In [143]:
print(texts[0].page_content)
print('-------------------')
print(texts[1].page_content)
print('-------------------')
print(texts[2].page_content)
print('-------------------')
print(texts[3].page_content)
print('-------------------')
print(texts[4].page_content)
print('-------------------')
print(texts[5].page_content)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans. Last year COVID-19 kept us apart. This year we are finally together again. Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. With a duty to one another to the American people to the Constitution. And with an unwavering resolve that freedom will always triumph over tyranny. Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. He met the Ukrainian people. From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. Groups of citizens blocking tanks with their bodies. Everyone from students t